# Link Recommendation for Reducing Polarization in Social Networks

## Basic Classes for Calculation

In [9]:
import networkx as nx
import numpy


class PolarizationGraph:
    polarization = 0

    def __init__(self, graph: nx.Graph, internal_opinions: list):
        self.nx_graph = graph
        # the internal_opinions list contains a dict for each node in the form {"value" : x, "polarization": y}
        self.internal_opinions = internal_opinions
        self.external_opinions = [
            {"value": x["value"], "polarization": 0} for x in internal_opinions]

    def setExternal(self, new_external: list):
        self.external_opinions = new_external


class PolarizationHandler:
    def calcExternalOpinions(graph: PolarizationGraph) -> list:
        expressed = []

        for iter in range(1000):
            for n in graph.nx_graph.nodes():
                node_name = n if isinstance(n, int) else n[0]
                my_internal = next(
                    x["polarization"] for x in graph.internal_opinions if x["value"] == node_name)
                my_neighbours = list(nx.neighbors(graph.nx_graph, n))
                external_neighbours = [
                    x["polarization"] for x in graph.external_opinions if x["value"] in my_neighbours]
                my_expressed = (my_internal + numpy.sum(external_neighbours)) / \
                    (1 + numpy.sum([1 for neigh in my_neighbours]))

                if len(expressed) != len(graph.external_opinions):
                    expressed.append(
                        {"value": node_name, "polarization": my_expressed})
                else:
                    my_set = next(
                        s for s in expressed if s["value"] == node_name)
                    my_set["polarization"] = my_expressed

            graph.setExternal(expressed)
            # print(PolarizationHandler.calcPolarization(graph))

        return expressed

    def calcPolarization(graph: PolarizationGraph) -> float:
        polArr = [x["polarization"] for x in graph.external_opinions]
        nbrNodes = nx.number_of_nodes(graph.nx_graph)
        return numpy.linalg.norm(polArr) / nbrNodes

class HeuristicsHandler:
    def getExtremeExpressed(graph: PolarizationGraph, nbr_of_nodes_to_neutralize: int) -> PolarizationGraph:
        external_opinions = graph.external_opinions
        
        sorted_list = sorted(external_opinions, key=
        
        iter = 0
        while (iter < nbr_of_nodes_to_neutralize):
            most_extreme_opinion = 0
            node_to_neutralize = 0
            for x in external_opinions:
                current_abs_opinion = abs(x["polarization"])
                if current_abs_opinion > most_extreme_opinion:
                    most_extreme_opinion = current_abs_opinion
                    node_to_neutralize = x["value"]
            external_opinions[node_to_neutralize - 1] = {"value": node_to_neutralize, "polarization": 0}
            graph.setExternal(PolarizationHandler.calcExternalOpinions(graph))
            external_opinions = graph.external_opinions
            iter+=1
        return graph
        

## Data Setup

In [11]:
import networkx as nx

KarateGraph = nx.read_gml("karate.gml", None)
BooksGraph = nx.read_gml("polbooks.gml", None)

ground_truth_as_sets = [{20, 1, 2, 22, 3, 5, 4, 7, 6, 8, 11, 13, 12, 14, 18, 10, 17},
                        {24, 26, 28, 33, 30, 34, 25, 32, 27, 21, 23, 29, 9, 31, 15, 16, 19}]

internal_karate = []
internal_books = []

for i in range(1, 35):
    polarization = -1 if i in ground_truth_as_sets[0] else 1
    internal_karate.append({"value": i, "polarization": polarization})

for n in BooksGraph.nodes.data():
    polarization = 0 if n[1]["value"] == "n" else - \
        1 if n[1]["value"] == "c" else 1
    internal_books.append({"value": n[0], "polarization": polarization})

KaratePolGraph = PolarizationGraph(KarateGraph, internal_karate)
BooksPolGraph = PolarizationGraph(BooksGraph, internal_books)

KaratePolGraph.setExternal(PolarizationHandler.calcExternalOpinions(KaratePolGraph))
print(PolarizationHandler.calcPolarization(KaratePolGraph))


for gr in [
    KaratePolGraph, 
    BooksPolGraph
]:
    exprOpinions = PolarizationHandler.calcExternalOpinions(gr)
    gr.setExternal(exprOpinions)
    gr.polarization = PolarizationHandler.calcPolarization(gr)
    print(gr.polarization)

KaratePolGraph = HeuristicsHandler.applyExtremeExpressed(KaratePolGraph, 6)
print(KaratePolGraph.polarization)

0.10221847999166145
0.10221847999166145
0.06431294911675486
0.10221847999166145
